In [1]:
! pip install langgraph langchain-google-genai pydantic langchain_core

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 2.6 MB/s eta 0:00:00


In [2]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver

In [3]:
from google.colab import userdata
api_key = userdata.get('GOOGLE_API_KEY')

In [4]:
model = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash"
    , api_key= api_key)

In [5]:
class PostState(TypedDict):

    topic: str
    post: str
    hashtags : str

In [6]:
def generate_post(state: PostState):
  prompt = f'generate a funny and engaging instagram thread post on the topic {state["topic"]}. Max 30 words '
  response = model.invoke(prompt).content

  return {'post': response}


In [7]:
def generate_hashtags(state: PostState):
  prompt = f'generate 3 to 5 hashtags on the topic {state["topic"]} for instagram thread post : {state["post"]}  , so that this post goes viral . just give has'
  response = model.invoke(prompt).content

  return {'hashtags': response}


In [8]:
graph = StateGraph(PostState)

graph.add_node('generate_post', generate_post)
graph.add_node('generate_hashtags', generate_hashtags)

graph.add_edge(START, 'generate_post')
graph.add_edge('generate_post', 'generate_hashtags')
graph.add_edge('generate_hashtags', END)

checkpointer = InMemorySaver()

workflow = graph.compile(checkpointer=checkpointer)

In [9]:
config1 = {"configurable": {"thread_id": "1"}}
workflow.invoke({'topic':'donald trump'}, config=config1)

{'topic': 'donald trump',
 'post': 'Imagine Trump\'s old tweets as a historical document. What\'s the wildest thing you\'d find? 🤣 Mine\'s "covfefe."',
 'hashtags': '#TrumpTweets\n#Covfefe\n#DigitalHistory\n#WildestTweets\n#SocialMediaArchive'}

In [10]:
workflow.get_state(config1)

StateSnapshot(values={'topic': 'donald trump', 'post': 'Imagine Trump\'s old tweets as a historical document. What\'s the wildest thing you\'d find? 🤣 Mine\'s "covfefe."', 'hashtags': '#TrumpTweets\n#Covfefe\n#DigitalHistory\n#WildestTweets\n#SocialMediaArchive'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f111573-561d-6a45-8002-f1a079f2373e'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2026-02-24T08:02:48.062792+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f111573-32b4-6c7b-8001-cd8264f70eb2'}}, tasks=(), interrupts=())

In [11]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'donald trump', 'post': 'Imagine Trump\'s old tweets as a historical document. What\'s the wildest thing you\'d find? 🤣 Mine\'s "covfefe."', 'hashtags': '#TrumpTweets\n#Covfefe\n#DigitalHistory\n#WildestTweets\n#SocialMediaArchive'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f111573-561d-6a45-8002-f1a079f2373e'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2026-02-24T08:02:48.062792+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f111573-32b4-6c7b-8001-cd8264f70eb2'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'topic': 'donald trump', 'post': 'Imagine Trump\'s old tweets as a historical document. What\'s the wildest thing you\'d find? 🤣 Mine\'s "covfefe."'}, next=('generate_hashtags',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f111573-32b4-6c7b-8001-cd8264f70eb2'}}, metadata={'source'

In [12]:
list(workflow.get_state_history(config1))[0]

StateSnapshot(values={'topic': 'donald trump', 'post': 'Imagine Trump\'s old tweets as a historical document. What\'s the wildest thing you\'d find? 🤣 Mine\'s "covfefe."', 'hashtags': '#TrumpTweets\n#Covfefe\n#DigitalHistory\n#WildestTweets\n#SocialMediaArchive'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f111573-561d-6a45-8002-f1a079f2373e'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2026-02-24T08:02:48.062792+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f111573-32b4-6c7b-8001-cd8264f70eb2'}}, tasks=(), interrupts=())

In [13]:
list(workflow.get_state_history(config1))[-1]

StateSnapshot(values={}, next=('__start__',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f111573-03e6-6e59-bfff-f3724fe9d0fd'}}, metadata={'source': 'input', 'step': -1, 'parents': {}}, created_at='2026-02-24T08:02:39.442066+00:00', parent_config=None, tasks=(PregelTask(id='e1566f0c-332e-bfb3-b90d-21b0db09801f', name='__start__', path=('__pregel_pull', '__start__'), error=None, interrupts=(), state=None, result={'topic': 'donald trump'}),), interrupts=())

In [14]:
list(workflow.get_state_history(config1))[-2]

StateSnapshot(values={'topic': 'donald trump'}, next=('generate_post',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f111573-03ed-6032-8000-0ca6fcaffd99'}}, metadata={'source': 'loop', 'step': 0, 'parents': {}}, created_at='2026-02-24T08:02:39.444558+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f111573-03e6-6e59-bfff-f3724fe9d0fd'}}, tasks=(PregelTask(id='28777576-3cb4-6d3d-ba37-24680dc4b8fd', name='generate_post', path=('__pregel_pull', 'generate_post'), error=None, interrupts=(), state=None, result={'post': 'Imagine Trump\'s old tweets as a historical document. What\'s the wildest thing you\'d find? 🤣 Mine\'s "covfefe."'}),), interrupts=())

# different thread

In [17]:
config2 = {"configurable": {"thread_id": "2"}}
workflow.invoke({'topic':'modi'}, config=config2)

{'topic': 'modi',
 'post': "Modi's got that 'always on' vibe! ✨ If you could ask him one non-political question, what would it be? Drop your funniest ideas below! 👇",
 'hashtags': 'Here are 3 to 5 hashtags for your Instagram Thread post:\n\n#ModiVibe\n#AskModi\n#NonPoliticalFun\n#FunnyQuestions\n#Modi'}

In [18]:
list(workflow.get_state_history(config2))

[StateSnapshot(values={'topic': 'modi', 'post': "Modi's got that 'always on' vibe! ✨ If you could ask him one non-political question, what would it be? Drop your funniest ideas below! 👇", 'hashtags': 'Here are 3 to 5 hashtags for your Instagram Thread post:\n\n#ModiVibe\n#AskModi\n#NonPoliticalFun\n#FunnyQuestions\n#Modi'}, next=(), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f11157e-ce50-66d2-8006-e4f0d6a9d787'}}, metadata={'source': 'loop', 'step': 6, 'parents': {}}, created_at='2026-02-24T08:07:55.945515+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f11157e-a357-623d-8005-3376ae39ac8c'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'topic': 'modi', 'post': "Modi's got that 'always on' vibe! ✨ If you could ask him one non-political question, what would it be? Drop your funniest ideas below! 👇", 'hashtags': 'Here are 3 to 5 hashtags for your post:\n\n#ModiThreads\n#NarendraModi\n#ModiStyle\